In [1]:
import re
import pandas as pd
from statistics import mean
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter

from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("../../data_Kaggle/train.csv")
print(train.shape)
print(train.columns)

(3947, 3)
Index(['Insult', 'Date', 'Comment'], dtype='object')


In [4]:
test = pd.read_csv("../../data_Kaggle/test_with_solutions.csv")
print(test.shape)
print(test.columns)

(2647, 4)
Index(['Insult', 'Date', 'Comment', 'Usage'], dtype='object')
